In [2]:
%matplotlib

import sys
sys.path.append('..')

from brian2 import defaultclock, ms, pA, nA, Hz, seed, mV, second
import numpy as np
from results import Results
import matplotlib.pyplot as plt

Using matplotlib backend: Qt5Agg


INFO       Cache size for target "cython": 13145 MB.
You can call "clear_cache('cython')" to delete all files from the cache or manually delete files in the "C:\Users\mikejseay\.cython\brian_extensions" directory. [brian2]


In [86]:
# targetFile = 'buonoEphys_2000_0p25_upPoisson_guessBuono2Weights2e3p025LogNormal_test_2021-08-13-14-55_results'  # very nice but not the right params exactly
# targetFile = 'buonoEphys2_2000_0p25_upPoisson_guessBuono2Weights2e3p025LogNormal2_test_2021-08-16-14-26_results'  # very nice
# targetFile = 'buonoEphys2_2000_0p25_upPoisson_guessBuono2Weights2e3p025LogNormal2_test_2021-08-16-14-35_results'  # very nice
# targetFile = 'buonoEphysBen1_2000_0p25_upPoisson_guessBuono2Weights2e3p025LogNormal_test_2021-08-16-16-19_results'  # very nice (ben idea worked)
# targetFile = 'buonoEphysBen1_2000_0p25_upPoisson_guessBuono2Weights2e3p025LogNormal_test_2021-08-16-16-46_results'  # another example
# targetFile = 'buonoEphysBen1_2000_0p25_upPoisson_guessBuono2Weights2e3p025LogNormal_test_2021-08-17-11-09_results'  # good

targetFile = 'buonoEphysBen1_2000_0p25_upPoisson_guessBuono2Weights2e3p025LogNormal_test_2021-08-17-14-13_results'  # no Exc2
# targetFile = 'buonoEphysBen1_2000_0p25_upPoisson_guessBuono2Weights2e3p025LogNormal_test_2021-08-17-14-16_results'  # yes Exc2
targetFolder = 'C:/Users/mikejseay/Documents/BrianResults/upPoisson'

In [87]:
R = Results()
R.init_from_file(targetFile, targetFolder)

print(R.p['simName'])

classicJercogEphysBuonoBen1


In [88]:
plt.close('all')

In [89]:
R.calculate_PSTH()
R.calculate_voltage_histogram(removeMode=True, useAllRecordedUnits=True)
R.calculate_upstates()
if len(R.ups) > 0:
    R.reshape_upstates()
    R.calculate_FR_in_upstates()
    infoStr = 'average FR in upstate for Exc: {:.2f}, Inh: {:.2f}, average upDur: {:.2f}'.format(R.upstateFRExcHist.mean(), R.upstateFRInhHist.mean(), R.upDurs.mean())
    print(infoStr)


R.calculate_voltage_histogram(removeMode=True)
R.reshape_upstates()

fig1, ax1 = plt.subplots(6, 1, num=1, figsize=(16, 9),
                         gridspec_kw={'height_ratios': [3, 1, 1, 1, 1, 1]},
                         sharex=True)
R.plot_spike_raster(ax1[0])  # uses RNG but with a separate random seed
R.plot_firing_rate(ax1[1])
ax1[1].set_ylim(0, 30)
R.plot_voltage_detail(ax1[2], unitType='Exc', useStateInd=0)
R.plot_updur_lines(ax1[2])
R.plot_voltage_detail(ax1[3], unitType='Inh', useStateInd=0)
R.plot_updur_lines(ax1[3])
R.plot_voltage_detail(ax1[4], unitType='Exc', useStateInd=1)
R.plot_updur_lines(ax1[4])
if 'useSecondPopExc' in R.p and R.p['useSecondPopExc']:
    R.plot_voltage_detail(ax1[5], unitType='Exc', useStateInd=2)
    R.plot_updur_lines(ax1[5])
ax1[5].set(xlabel='Time (s)')
R.plot_voltage_histogram_sideways(ax1[2], 'Exc')
R.plot_voltage_histogram_sideways(ax1[3], 'Inh')
fig1.suptitle(infoStr)

average FR in upstate for Exc: 2.31, Inh: 4.73, average upDur: 2.07


Text(0.5, 0.98, 'average FR in upstate for Exc: 2.31, Inh: 4.73, average upDur: 2.07')

In [21]:
R.calculate_upFR_units()

In [22]:
R.upstateFRExcUnits.shape

(8, 1600)

In [27]:
R.p['nUnitsToSpike']

100

In [28]:
R.p['nUnitsSecondPopExc']

100

In [58]:
frInp = R.upstateFRExcUnits[:, :R.p['nUnitsToSpike']].mean(0)  # input pop
frEx2 = R.upstateFRExcUnits[:, R.p['nUnitsToSpike']:R.p['nUnitsToSpike'] + R.p['nUnitsSecondPopExc']].mean(0)  # secondary pop
frEx1 = R.upstateFRExcUnits[:, 200:].mean(0)  # normal pop
frInh = R.upstateFRInhUnits.mean(0)

In [59]:
frInpHat = frInp.mean()
frEx2Hat = frEx2.mean()
frEx1Hat = frEx1.mean()
frInhHat = frInh.mean()

In [60]:
frInpSTD = frInp.std()
frEx2STD = frEx1.std()
frEx1STD = frEx1.std()
frInhSTD = frInh.std()

In [61]:
print(frInpHat, frEx2Hat, frEx1Hat, frInhHat, )

2.6257324199438994 0.5135641858576404 2.960596197631221 6.757260244565425


In [62]:
print(frInpSTD, frEx2STD, frEx1STD, frInhSTD, )

0.48798605018010816 0.49950287580926406 0.49950287580926406 2.9756498702977696


In [49]:
import seaborn as sns
import pandas as pd

In [65]:
frInpDF = pd.DataFrame(frInp, columns=('FR',))
frEx2DF = pd.DataFrame(frEx2, columns=('FR',))
frEx1DF = pd.DataFrame(frEx1, columns=('FR',))
frInhDF = pd.DataFrame(frInh, columns=('FR',))

In [66]:
frInpDF['unitType'] = 'Inp'
frEx2DF['unitType'] = 'Ex2'
frEx1DF['unitType'] = 'Ex1'
frInhDF['unitType'] = 'Inh'

In [67]:
frDF = pd.concat((frInpDF, frEx2DF, frEx1DF, frInhDF, ))

In [68]:
frDF

,FR,unitType
0,3.573941,Inp
1,3.302398,Inp
2,2.217661,Inp
3,2.385036,Inp
4,2.397402,Inp
...,...,...
395,6.062659,Inh
396,4.238428,Inh
397,10.620160,Inh
398,2.507565,Inh


In [75]:
sns.boxplot(x='unitType', y='FR', data=frDF, width=.6, palette="vlag")
# sns.stripplot(x="unitType", y="FR", data=frDF, size=4, color=".3", linewidth=0)

In [76]:
sns.swarmplot(x='unitType', y='FR', data=frDF)

In [22]:
# get the QTCore PyRect object
geom = mngr.window.geometry()

x,y,dx,dy = geom.getRect()

print(x,y,dx,dy)

1997 83 640 532


In [ ]:
mngr = plt.get_current_fig_manager()
# to put it into the upper left corner for example:
# mngr.window.setGeometry(50,100,640, 545)
targetFile = ''